In [45]:
import numpy as np 
import matplotlib.pyplot as plt 
from sparse_rf.algs.core import min_wl2, min_l2, min_l1
from sparse_rf.modules import make_X, make_W, make_A
from sparse_rf.util import *
from sparse_rf.util.fns import *
from functools import partial
import random
import gc

In [88]:
m = 1000
d = 100
q = 2
N = 10000
active = identity

ratio_train = 0.7

In [89]:
X = make_X(d, m, dist=uniform)
X_train = X[:int(m*ratio_train), :]
X_test = X[int(m*ratio_train):, :]

y = np.array(list(map(fn2, X)))
y_train = y[:int(m*ratio_train)]
y_test = y[int(m*ratio_train):]

In [90]:
W = make_W(d, q, N=N, dist=partial(normal, stdev=1))
A_train = make_A(X_train, W, active=active)
A_test = make_A(X_test, W, active=active)

In [91]:
c_l1 = min_l1(A_train, y_train, sigma=0)
c_l2 = min_l2(A_train, y_train)

In [92]:
np.linalg.norm(A_test@c_l1-y_test)/ np.linalg.norm(y_test)

0.8679823605553648

In [93]:
np.linalg.norm(A_test@c_l2-y_test)/ np.linalg.norm(y_test)

0.8773150181307939